# QAOA Benchmark: k-hot, SWAP, XY, X Mixers

This notebook benchmarks QAOA performance on random Hamiltonians
using k-hot initial states and different mixers.

---

## How To Run This Notebook

**1. Build the C simulator**

If you’re running QOKit from source, compile the C backend:

```bash
cd qokit/fur/c/csim/src
make clean
make CC=gcc-15
cp ../libcsim.dylib ../libcsim.so


In [ ]:
# QAOA Benchmark Notebook
# ------------------------

import numpy as np
import scipy.optimize
import pandas as pd
import matplotlib.pyplot as plt

from qokit.qaoa_objective import get_qaoa_objective

print("All libraries imported.")

In [ ]:
# Benchmark Parameters

N = 10              # Number of qubits
K = 3               # Hamming weight
p = 3               # QAOA layers
num_terms = 50      # Hamiltonian terms
n_trials = 20       # Random restarts

print(f"Benchmark setup:\nN = {N}, K = {K}, p = {p}")

In [ ]:
# Generate random Hamiltonian

np.random.seed(42)

terms = []
for _ in range(num_terms):
    i, j = np.random.choice(N, size=2, replace=False)
    coeff = np.random.uniform(-1, 1)
    terms.append((coeff, [int(i), int(j)]))

print(f"Random Hamiltonian generated with {len(terms)} terms.")

In [ ]:
# Benchmark loop for X, XY and SWAP

mixers_to_test = ["x", "xy", "swap"]

results = []

for m in mixers_to_test:
    best_cost = float("inf")
    all_costs = []
    nfev_list = []
    
    for trial in range(n_trials):
        theta0 = np.random.uniform(-np.pi, np.pi, size=2*p)
        
        # Determine k-hot argument
        if m in ["xy", "swap"]:
            f = get_qaoa_objective(
                N=N,
                terms=terms,
                mixer=m,
                k_hot=K
            )
            khot_flag = True
        else:
            f = get_qaoa_objective(
                N=N,
                terms=terms,
                mixer=m,
                k_hot=None
            )
            khot_flag = False
        
        res = scipy.optimize.minimize(
            f,
            theta0,
            method="COBYLA",
            options={"maxiter": 500}
        )
        
        all_costs.append(res.fun)
        nfev_list.append(res.nfev)
        
        if res.fun < best_cost:
            best_cost = res.fun
    
    avg_cost = np.mean(all_costs)
    std_cost = np.std(all_costs)
    avg_nfev = np.mean(nfev_list)
    
    results.append({
        "Mixer": m,
        "k-hot Used": khot_flag,
        "Best Final Cost": best_cost,
        "Average Final Cost": avg_cost,
        "Std Dev": std_cost,
        "Average Function Evaluations": avg_nfev
    })

df = pd.DataFrame(results)
print(df)


In [ ]:
# Plot average cost comparison

plt.figure(figsize=(6,4))
bars = plt.bar(df["Mixer"], df["Average Final Cost"], color=['lightgreen', 'skyblue', 'salmon'])
plt.ylabel("Average Final Cost")
plt.title(f"QAOA Benchmark N={N}, k={K}, p={p}")
plt.show()


In [ ]:
# Display table nicely

print("=== Benchmark Summary ===")
display(df)